In [1]:
# import pandas as pd

# df1 = pd.read_csv('/kaggle/input/thaisum-100-1024-nlpfinal/test-100-1024.csv')
# df2 = pd.read_csv('/kaggle/input/gemini-sum/gemini_summaries.csv')
# print(list(df1['body']) == list(df2['body']))

In [4]:
!pip install deepeval aiobotocore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 88.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.29
    Uninstalling botocore-1.37.29:
      Successfully uninstalled botocore-1.37.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.37.29 requires botocore<1.38.0,>=1.37.29, but you have botocore 1.37.3 which is incompatible.
s3transfer 0.11.4 requires botocore<2.0a.0,>=1.37.4, but you have botocore 1.37.3 which is incompatible.


In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("Gemini_2.0_flash_api_key")

In [6]:
# This is the original text to be summarized
input = """
The 'coverage score' is calculated as the percentage of assessment questions
for which both the summary and the original document provide a 'yes' answer. This
method ensures that the summary not only includes key information from the original
text but also accurately represents it. A higher coverage score indicates a
more comprehensive and faithful summary, signifying that the summary effectively
encapsulates the crucial points and details from the original content.
"""

# This is the summary, replace this with the actual output from your LLM application
actual_output="""
The coverage score quantifies how well a summary captures and
accurately represents key information from the original text,
with a higher score indicating greater comprehensiveness.
"""

In [7]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric

from deepeval.models import GeminiModel
from deepeval.metrics import AnswerRelevancyMetric

model = GeminiModel(
    model_name="gemini-2.0-flash",
    api_key=API_KEY,
    # temperature=0
)


test_case = LLMTestCase(input=input, actual_output=actual_output)
metric = SummarizationMetric(
    threshold=0.5,
    model=model,
    # assessment_questions=[
    #     "Is the coverage score based on a percentage of 'yes' answers?",
    #     "Does the score ensure the summary's accuracy with the source?",
    #     "Does a higher score mean a more comprehensive summary?"
    # ],
    n=10
)

# To run metric as a standalone
# metric.measure(test_case)
# print(metric.score, metric.reason)

result = evaluate(test_cases=[test_case], metrics=[metric])

✨ You're running DeepEval's latest Summarization Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:05,  5.10s/test case]



Metrics Summary

  - ❌ Summarization (score: 0.4, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.40 because the summary fails to answer several questions that the original text can answer, indicating a lack of detail and faithfulness to the original content. However, there are no contradictions or extra information, which is commendable., error: None)

For test case:

  - input: 
The 'coverage score' is calculated as the percentage of assessment questions
for which both the summary and the original document provide a 'yes' answer. This
method ensures that the summary not only includes key information from the original
text but also accurately represents it. A higher coverage score indicates a
more comprehensive and faithful summary, signifying that the summary effectively
encapsulates the crucial points and details from the original content.

  - actual output: 
The coverage score quantifies how well a summary captures and
accurately represe

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

In [10]:
import re
import ast

# result.test_results[0].metrics_data[0].score
verbose_logs = result.test_results[0].metrics_data[0].verbose_logs

# Use regex to find the list of assessment questions
match = re.search(r'Assessment Questions:\n(\[.*?\])', verbose_logs, re.DOTALL)
if match:
    questions_str = match.group(1)
    questions = ast.literal_eval(questions_str)

print(questions)

['Is the coverage score calculated based on assessment questions?', "Does the summary provide a 'yes' answer for some assessment questions?", "Does the original document provide a 'yes' answer for some assessment questions?", 'Is the coverage score a percentage?', 'Does the method ensure the summary includes key information?', 'Does the method ensure the summary accurately represents the original text?', 'Does a higher coverage score indicate a more comprehensive summary?', 'Does a higher coverage score indicate a more faithful summary?', 'Does the summary effectively encapsulate crucial points?', 'Does the summary effectively encapsulate details from the original content?']


In [ ]:
# Fn in batch

In [11]:
# input = ["""
# The 'coverage score' is calculated as the percentage of assessment questions
# for which both the summary and the original document provide a 'yes' answer. This
# method ensures that the summary not only includes key information from the original
# text but also accurately represents it. A higher coverage score indicates a
# more comprehensive and faithful summary, signifying that the summary effectively
# encapsulates the crucial points and details from the original content.
# """ for _ in range(10)] 

# # This is the summary, replace this with the actual output from your LLM application
# actual_output=["""
# The coverage score quantifies how well a summary captures and
# accurately represents key information from the original text,
# with a higher score indicating greater comprehensiveness.
# """ for _ in range(10)]

# my_pairs = [("""
# The 'coverage score' is calculated as the percentage of assessment questions
# for which both the summary and the original document provide a 'yes' answer. This
# method ensures that the summary not only includes key information from the original
# text but also accurately represents it. A higher coverage score indicates a
# more comprehensive and faithful summary, signifying that the summary effectively
# encapsulates the crucial points and details from the original content.
# """, """
# The coverage score quantifies how well a summary captures and
# accurately represents key information from the original text,
# with a higher score indicating greater comprehensiveness.
# """) for _ in range(2)]
import pandas as pd

df = pd.read_csv('/kaggle/input/gemini-sum/gemini_summaries.csv')
my_pairs = list(df[['body', 'generated']].itertuples(index=False, name=None))

In [12]:
import re
import ast  # safer than eval for parsing list strings

def extract_assessment_questions(evaluation_result):
    try:
        verbose_logs = evaluation_result.test_results[0].metrics_data[0].verbose_logs

        # Use regex to find the list of assessment questions
        match = re.search(r'Assessment Questions:\n(\[.*?\])', verbose_logs, re.DOTALL)
        if match:
            questions_str = match.group(1)
            questions = ast.literal_eval(questions_str)
            
            return questions
        else:
            return []
    except Exception as e:
        print(f"Error extracting questions: {e}")
        return []

In [10]:
ass_questions = []

In [13]:
import time
import json
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric
from deepeval.models import GeminiModel

def batch_summarization_score(
    input_output_pairs,
    api_key,
    sleep_time=20,
    max_retries=3,
    retry_delay=60
):
    """
    Evaluates summarization quality for each pair using the Gemini model with deepeval,
    and handles batching to respect rate limits.

    Args:
        input_output_pairs: List of tuples (input, actual_output)
        api_key: Gemini API key string
        batch_size: Number of evaluations before sleeping to avoid rate limits
        sleep_time: Sleep time in seconds between batches
        max_retries: Max retry attempts per pair
        retry_delay: Seconds between retries

    Returns:
        List of dicts: {input, actual_output, score, reason}
    """
    
    # ass_questions = None
    
    model = GeminiModel(
        model_name="gemini-2.0-flash",
        api_key=api_key,
    )

    metric = SummarizationMetric(
        threshold=0.5,
        model=model,
        # assessment_questions=['Is the coverage score calculated based on assessment questions?', "Does the summary provide a 'yes' answer for some assessment questions?", "Does the original document provide a 'yes' answer for some assessment questions?", 'Is the coverage score a percentage?', 'Does the method ensure the summary includes key information?', 'Does the method ensure the summary accurately represents the original text?', 'Does a higher coverage score indicate a more comprehensive summary?', 'Does a higher coverage score indicate a more faithful summary?', 'Does the summary effectively encapsulate crucial points?', 'Does the summary effectively encapsulate details from the original content?'],
        # include_reason=False,
        n=10
    )

    results = []

    for idx, (inp, out) in enumerate(input_output_pairs):
        test_case = LLMTestCase(input=inp, actual_output=out)
        attempt = 0
        success = False

        while attempt < max_retries and not success:
            try:
                result = evaluate(test_cases=[test_case], metrics=[metric])
                results.append(result)
                print('Assessment Questions')
                for i, q in enumerate(extract_assessment_questions(result)):
                    print(f"{i+1}. {q}")
                
                # first time ass_questions
                ass_questions.append(extract_assessment_questions(result))
                
                success = True
            except Exception as e:
                print(f"[Attempt {attempt + 1}] Error on item {idx + 1}: {e}")
                attempt += 1
                if attempt < max_retries:
                    time.sleep(retry_delay)
                else:
                    results.append({
                        "input": inp,
                        "actual_output": out,
                        "score": None,
                        "reason": f"Failed after {max_retries} attempts: {e}"
                    })
        
        # Sleep after every items
        time.sleep(sleep_time)
        break

    # # save ass_questions as a file
    # with open("assessment_questions.json", "w", encoding="utf-8") as f:
    #     json.dump(ass_questions, f, ensure_ascii=False, indent=2)
        
    return results

In [14]:
results = batch_summarization_score(input_output_pairs=my_pairs, api_key=API_KEY)
# for r in results:
#     if 'test_results' in r:
#         print(r.test_results[0].metrics_data[0].score)
#     else: print('None')

✨ You're running DeepEval's latest Summarization Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:06,  6.38s/test case]



Metrics Summary

  - ✅ Summarization (score: 0.6, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.60 because the summary fails to address key details such as the police chief's involvement, the scope of tourist police services, the safety of Chiang Mai for tourists, and the specific measures taken by the Chiang Mai police to protect tourists., error: None)

For test case:

  - input: ศูนย์ข่าวภาคเหนือ -12 เม.ย.2548ตำรวจท่องเที่ยวเชียงใหม่พาอาสาสมัครชาวต่างชาติออกบริการนักท่องเที่ยว ลบภาพข่าวผู้ก่อการร้ายหลบหนีเข้าเชียงใหม่ พ.ต.ท.ณัฐวุฒิ โชติกาญจนวัฒน์ ผู้กำกับการสถานีตำรวจท่องเที่ยว จ.เชียงใหม่ ได้นำทีมตำรวจท่องเที่ยวและอาสาสมัครชาวต่างประเทศ ออกให้บริการแก่นักท่องเที่ยวชาวไทยและชาวต่างประเทศที่เดินทางมาร่วมงานสงกรานต์เชียงใหม่ในปีนี้ รวมทั้งได้หามาตรการในการักษาความปลอดภัยในชีวิตและทรัพย์สิน โดยได้จัดเจ้าหน้าที่ตำรวจ ออกปฏิบัติหน้าที่เพื่อป้องกันการก่ออาชญากรรม และการก่อความเดือดร้อนรำคาญที่เกิดขึ้นกับนักท่องเที่ยวในย่านประตูท่าแพ และจุดต่าง

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Assessment Questions
1. Is the Chiang Mai Tourist Police providing services to tourists?
2. Did the Chiang Mai Tourist Police work with foreign volunteers?
3. Did the police chief, Nattawut Chotikarnjanawat, lead the tourist police team?
4. Did the tourist police provide services to both Thai and foreign tourists?
5. Were safety measures implemented for tourists during Songkran in Chiang Mai?
6. Did the police officers perform duties to prevent crime?
7. Did the police address concerns about terrorists in Chiang Mai?
8. Was the news about terrorists in Chiang Mai found to be untrue?
9. Is it safe to travel in Chiang Mai according to the text?
10. Did the Chiang Mai police take measures to ensure the safety of tourists' lives and property?
[Attempt 1] Error on item 1: name 'ass_questions' is not defined


...

✨ You're running DeepEval's latest Summarization Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:06,  6.52s/test case]



Metrics Summary

  - ✅ Summarization (score: 0.7, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.70 because the summary omits details about the police chief leading the tourist police team, the tourist police providing services to both Thai and foreign tourists, and the police chief's assurance about the safety of tourism in Chiang Mai. However, there are no contradictions or extra information, indicating a reasonably good summary., error: None)

For test case:

  - input: ศูนย์ข่าวภาคเหนือ -12 เม.ย.2548ตำรวจท่องเที่ยวเชียงใหม่พาอาสาสมัครชาวต่างชาติออกบริการนักท่องเที่ยว ลบภาพข่าวผู้ก่อการร้ายหลบหนีเข้าเชียงใหม่ พ.ต.ท.ณัฐวุฒิ โชติกาญจนวัฒน์ ผู้กำกับการสถานีตำรวจท่องเที่ยว จ.เชียงใหม่ ได้นำทีมตำรวจท่องเที่ยวและอาสาสมัครชาวต่างประเทศ ออกให้บริการแก่นักท่องเที่ยวชาวไทยและชาวต่างประเทศที่เดินทางมาร่วมงานสงกรานต์เชียงใหม่ในปีนี้ รวมทั้งได้หามาตรการในการักษาความปลอดภัยในชีวิตและทรัพย์สิน โดยได้จัดเจ้าหน้าที่ตำรวจ ออกปฏิบัติหน้าที่เพื่อป้องกันการก่

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Assessment Questions
1. Is the Chiang Mai Tourist Police providing services to tourists?
2. Did the Chiang Mai Tourist Police work with foreign volunteers?
3. Did the police chief, Nattawut Chotikarnjanawat, lead the tourist police team?
4. Did the tourist police provide services to both Thai and foreign tourists?
5. Were safety measures implemented during the Songkran festival in Chiang Mai?
6. Did the police officers perform duties to prevent crime?
7. Did the police address concerns about terrorists in Chiang Mai?
8. Was the news about terrorists in Chiang Mai found to be untrue?
9. Did the police chief assure that tourism in Chiang Mai is safe?
10. Did the Chiang Mai police take measures to ensure the safety of tourists' lives and property?
[Attempt 3] Error on item 1: name 'ass_questions' is not defined


In [25]:
scores = []
for r in results:
    try:    
        scores.append(r.test_results[0].metrics_data[0].score)
    except:
        scores.append(0)

In [26]:
sdf = pd.DataFrame(scores)
sdf["questions"] = ass_questions 
sdf.to_csv("gemini_scores.csv", index=False)

ValueError: Length of values (98) does not match length of index (100)

In [ ]:
for (s1,s2) in my_pairs:
    if len(s1) < 200:
        print(len(s1))